# Projeto: Construindo um mecanismo de busca semântica

Um motor de pesquisa semântica funcional que demonstra:

- Conhecimento especializado: escolha conteúdos que compreende para poder avaliar a qualidade da pesquisa
- Comparação de fragmentação: teste diferentes estratégias e veja qual funciona melhor para o seu tipo de conteúdo
- Compreensão semântica real: pesquise por conceito, tema ou significado, em vez de palavras-chave exatas
- Informações práticas: descubra o que torna a fragmentação eficaz na sua área específica

In [ ]:
!pip install -U sentence-transformers transformers qdrant-client llama-index-core llama-index-embeddings-huggingface -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.2/377.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 103.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 10.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [ ]:
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient, models
import os
from dotenv import load_dotenv

load_dotenv('/content/qdrant.env') # Carrega variáveis do arquivo .env

qdrant_url = os.getenv("QDRANT_URL")
qdrant_api_key = os.getenv("QDRANT_API_KEY")

client = QdrantClient(url=qdrant_url, api_key=qdrant_api_key)

encoder = SentenceTransformer("all-MiniLM-L6-v2")

Sucesso! URL carregada: https://62f4713c-61df-4137-ab35-d40676003640.us-east4-0.gcp.cloud.qdrant.io:6333


In [ ]:
my_dataset = [
    {
        "title": "Projeto Holhooja",
        "description": """O projeto concentra suas investigações na concepção de uma solução automatizada que
        aborde as tecnologias da Internet das Coisas (IoT) e na elaboração de um sistema online
        direcionado ao controle de acesso de ambientes laboratoriais. Este protótipo do sistema
        integrado foi desenvolvido para o Laboratório de Práticas Autônomas (LPA) e tem como
        principal atribuição a administração do registro de acesso do LPA, promovendo aprimoramentos
        em termos de praticidade e controle do uso, ao mesmo tempo em que realiza
        a verificação do tempo de permanência de cada um dos alunos cadastrados, facilitando
        a geração de certificados de horas complementares. Paralelamente ao sistema de controle
        de acesso, o escopo do projeto abrange a implementação de um sistema de automação
        no laboratório. Este sistema, em conjunto com o controle de acesso, visa disponibilizar a
        abertura do ambiente, gerenciar os dispositivos de ar condicionado e iluminação, forne-
        cendo informações em tempo real sobre as condições do laboratório.""",
        "area": "IoT",
        "subarea": "Automação",
        "Nível": "Graduação",
        "Instituição": "IFTO"
    },
]

In [ ]:
def fixed_size_chunks(text, chunk_size=100, overlap=20):
    """Split text into fixed-size chunks with overlap"""
    words = text.split()
    chunks = []

    for i in range(0, len(words), chunk_size - overlap):
        chunk_words = words[i:i + chunk_size]
        if chunk_words:  # Only add non-empty chunks
            chunks.append(' '.join(chunk_words))

    return chunks

def sentence_chunks(text, max_sentences=3):
    """Group sentences into chunks"""
    import re
    sentences = re.split(r'[.!?]+', text)
    sentences = [s.strip() for s in sentences if s.strip()]

    chunks = []
    for i in range(0, len(sentences), max_sentences):
        chunk_sentences = sentences[i:i + max_sentences]
        if chunk_sentences:
            chunks.append('. '.join(chunk_sentences) + '.')

    return chunks

def paragraph_chunks(text):
    """Split by paragraphs or double line breaks"""
    chunks = [chunk.strip() for chunk in text.split('\n\n') if chunk.strip()]
    return chunks if chunks else [text]  # Fallback to full text

In [ ]:
collection_name = "day1_semantic_search"

if client.collection_exists(collection_name=collection_name):
    client.delete_collection(collection_name=collection_name)

# Ccria uma coleção com 3 vetores nomeados, cada um deles implementando uma estratégia de segmentação
client.create_collection(
    collection_name=collection_name,
    vectors_config={
        "fixed": models.VectorParams(size=384, distance=models.Distance.COSINE),
        "sentence": models.VectorParams(size=384, distance=models.Distance.COSINE),
        "paragraph": models.VectorParams(size=384, distance=models.Distance.COSINE),
    },
)

# indexa os campos para filtragem (metadados)
client.create_payload_index(
    collection_name=collection_name,
    field_name="chunk_strategy",
    field_schema=models.PayloadSchemaType.KEYWORD,
)

points = []
point_id = 0

for item in my_dataset:
    description = item["description"]

    # processa cada uma das estratégias de segmentação
    strategies = {
        "fixed": fixed_size_chunks(description),
        "sentence": sentence_chunks(description),
        "paragraph": paragraph_chunks(description),
    }

    for strategy_name, chunks in strategies.items():
        for chunk_idx, chunk in enumerate(chunks):
            # Ccria o vetor desse segmento
            vectors = {strategy_name: encoder.encode(chunk).tolist()}

            points.append(
                models.PointStruct(
                    id=point_id,
                    vector=vectors,
                    payload={
                        **item,  # inclui metadados originais
                        "chunk": chunk,
                        "chunk_strategy": strategy_name,
                        "chunk_index": chunk_idx,
                    },
                )
            )
            point_id += 1

client.upload_points(collection_name=collection_name, points=points)
print(f"Uploaded {len(points)} chunks across three strategies")

Uploaded 5 chunks across three strategies


Como entrar na sala de estudos especializada

Monitoramento de quem usa a sala e por quanto tempo.

Ligar luzes e climatização de forma remota.

Geração automática de documentos de atividades extracurriculares.

Solução para administrar o uso de um espaço compartilhado.

O que o sistema faz quando alguém chega?

In [ ]:
def compare_search_results(query):
    """Comparando resultados"""
    print(f"Query: '{query}'\n")

    for strategy in ["fixed", "sentence", "paragraph"]:
        results = client.query_points(
            collection_name=collection_name,
            query=encoder.encode(query).tolist(),
            using=strategy,
            limit=3,
        )

        print(f"--- {strategy.upper()} CHUNKING ---")
        for i, point in enumerate(results.points, 1):
            print(f"{i}. {point.payload['title']}")
            print(f"   Score: {point.score:.3f}")
            print(f"   Chunk: {point.payload['chunk'][:80]}...")
        print()


test_queries = [
    "Como entrar na sala de estudos",
    "Ligar luzes e climatização de forma remota",
    "Solução para administrar o uso de um espaço compartilhado.",
]

for query in test_queries:
    compare_search_results(query)

Query: 'Como entrar na sala de estudos'

--- FIXED CHUNKING ---
1. Projeto Holhooja
   Score: 0.429
   Chunk: do tempo de permanência de cada um dos alunos cadastrados, facilitando a geração...
2. Projeto Holhooja
   Score: 0.348
   Chunk: O projeto concentra suas investigações na concepção de uma solução automatizada ...

--- SENTENCE CHUNKING ---
1. Projeto Holhooja
   Score: 0.466
   Chunk: Este sistema, em conjunto com o controle de acesso, visa disponibilizar a
      ...
2. Projeto Holhooja
   Score: 0.358
   Chunk: O projeto concentra suas investigações na concepção de uma solução automatizada ...

--- PARAGRAPH CHUNKING ---
1. Projeto Holhooja
   Score: 0.358
   Chunk: O projeto concentra suas investigações na concepção de uma solução automatizada ...

Query: 'Ligar luzes e climatização de forma remota'

--- FIXED CHUNKING ---
1. Projeto Holhooja
   Score: 0.286
   Chunk: O projeto concentra suas investigações na concepção de uma solução automatizada ...
2. Projeto Holhooja
   S

In [ ]:
def analyze_chunking_effectiveness():
    print("Análise das estratégias de segmentação")
    print("=" * 40)

    # Get chunk statistics for each strategy
    for strategy in ["fixed", "sentence", "paragraph"]:
        # Count chunks per strategy
        results = client.scroll(
            collection_name=collection_name,
            scroll_filter=models.Filter(
                must=[
                    models.FieldCondition(
                        key="chunk_strategy", match=models.MatchValue(value=strategy)
                    )
                ]
            ),
            limit=100,
        )

        chunks = results[0]
        chunk_sizes = [len(chunk.payload["chunk"]) for chunk in chunks]

        print(f"\n{strategy.upper()} STRATEGY:")
        print(f"  Total de segmentações: {len(chunks)}")
        print(f"  Tamanho médio da segmentação: {sum(chunk_sizes)/len(chunk_sizes):.0f} chars")
        print(f"  Tamanho do intervalo: {min(chunk_sizes)}-{max(chunk_sizes)} chars")


analyze_chunking_effectiveness()

Análise das estratégias de segmentação

FIXED STRATEGY:
  Total de segmentações: 2
  Tamanho médio da segmentação: 574 chars
  Tamanho do intervalo: 485-664 chars

SENTENCE STRATEGY:
  Total de segmentações: 2
  Tamanho médio da segmentação: 549 chars
  Tamanho do intervalo: 244-854 chars

PARAGRAPH STRATEGY:
  Total de segmentações: 1
  Tamanho médio da segmentação: 1099 chars
  Tamanho do intervalo: 1099-1099 chars
